# Challenge

Another approach to identifying fraudulent transactions is to look for outliers in the data. Standard deviation or quartiles are often used to detect outliers. Using this starter notebook, code two Python functions:

* One that uses standard deviation to identify anomalies for any cardholder.

* Another that uses interquartile range to identify anomalies for any cardholder.

## Identifying Outliers using Standard Deviation

In [367]:
# Initial imports
import pandas as pd
import numpy as np
import random
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


In [368]:
# Create a connection to the database
load_dotenv('../../.env')
db_url = os.getenv("db_url")
engine = create_engine(db_url)

query = """
        select t.*, c.card_holder_id
        from transaction t
        join credit_card c on t.card = c.card
        """

df = pd.read_sql(query, engine)
df.head()

,id,date,amount,card,id_merchant,card_holder_id
0,222,2018-01-01 21:35:10,6.22,3.561954e+15,69,13
1,2045,2018-01-01 21:43:12,3.83,5.135838e+15,85,13
2,395,2018-01-01 22:41:21,9.61,2.131939e+14,82,10
3,3309,2018-01-01 23:13:30,19.03,4.263694e+15,5,4
4,567,2018-01-01 23:15:10,2.95,4.498003e+12,64,18


In [369]:
# Write function that locates outliers using standard deviation
def transaction_tracker(card_holder_id):
    return df.loc[df['card_holder_id']==card_holder_id, 'amount']

def outliers(card_holder_id):
    outliers_df =transaction_tracker(card_holder_id)
    return pd.DataFrame(outliers_df[outliers_df> outliers_df.mean()+3*outliers_df.std()])

In [370]:
# Find anomalous transactions for 3 random card holders
rand_card_holders = np.random.randint(1,26,3)
for card_holder_id in rand_card_holders:
    if len(outliers(card_holder_id)) == 0:
        print(f"Card holder {card_holder_id} has no outlier transactions.")
    else:
        print(f"Card holder {card_holder_id} has outlier transactions as below:\n{outliers(card_holder_id)}.")

Card holder 14 has no outlier transactions.
Card holder 10 has no outlier transactions.
Card holder 12 has outlier transactions as below:
      amount
15    1031.0
219   1678.0
708   1530.0
1673  1102.0
1729  1592.0
1745  1108.0
2549  1075.0
3171  1123.0
3187  1802.0.


## Identifying Outliers Using Interquartile Range

In [371]:
# Write a function that locates outliers using interquartile range
def outliers_iqr(card_holder_id):
    outliers_df = transaction_tracker(card_holder_id)
    IQR_threshold = np.quantile(outliers_df, .75)+(np.quantile(outliers_df, .75)-np.quantile(outliers_df, .25))*1.5
    return pd.DataFrame(outliers_df[outliers_df> IQR_threshold])

In [372]:
# Find anomalous transactions for 3 random card holders


In [373]:
for card_holder_id in rand_card_holders:
    if len(outliers_iqr(card_holder_id)) == 0:
        print(f"Card holder {card_holder_id} has no outlier transactions.")
    else:
        print(f"Card holder {card_holder_id} has outlier transactions as below:\n{outliers_iqr(card_holder_id)}.")


Card holder 14 has no outlier transactions.
Card holder 10 has no outlier transactions.
Card holder 12 has outlier transactions as below:
      amount
15    1031.0
219   1678.0
708   1530.0
795    852.0
1673  1102.0
1729  1592.0
1745  1108.0
2549  1075.0
3153   233.0
3171  1123.0
3187  1802.0
3331   748.0.
